# make_piano_examples

Quick notebook to compose sound examples for my planned blog post about piano synthesis.

In [ ]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal

from IPython.display import Audio

In [ ]:
# Spectrogram display, copied from piano_heterodyne.

from scipy import fft

def dB(x):
    return 20 * np.log10(np.abs(x))

def spectrogram(x, fft_len=1024, win_len=None, hop_len=None, window_fn=np.hanning, sr=1, f_max=None, title=None):
    if win_len is None:
        win_len = fft_len
    if hop_len is None:
        hop_len = win_len // 2
    # Window.
    prepad_len = (fft_len - win_len) // 2
    win = np.hstack([np.zeros(prepad_len), window_fn(win_len), np.zeros(fft_len - win_len - prepad_len)])
    # Frame.
    frame_indices = np.arange(0, len(x) - win_len, hop_len)[:, np.newaxis] + np.arange(win_len)[np.newaxis, :]
    x_chunks_windowed = x[frame_indices] * win[np.newaxis, :]
    # Transform.
    stft_mag_db = dB(fft.fft(x_chunks_windowed, n=fft_len)[:, :(fft_len // 2 + 1)])
    num_frames, num_bins = stft_mag_db.shape
    t_base = np.arange(num_frames) * hop_len / sr
    f_base = np.arange(num_bins) * sr / fft_len
    plt.imshow(stft_mag_db.T, aspect='auto', origin='lower', extent=[t_base[0], t_base[-1], f_base[0], f_base[-1]])
    plt.clim(np.max(stft_mag_db) + [-80, 0])
    #plt.ylim([0, f_max / sr * fft_len])
    if f_max:
        plt.ylim([0, f_max])
    #x_times = np.arange(0, len(x) / sr, 0.25)
    #y_freqs = np.arange(0, 10000, 1000)
    #plt.xticks(x_times * sr / hop_len, x_times)
    #plt.yticks(y_freqs / sr * fft_len, y_freqs)
    plt.colorbar(label='level / dB')
    plt.xlabel('time / sec')
    plt.ylabel('freq / kHz')
    if title:
        plt.title(title)


In [ ]:
# Load in the original piano samples, copied from piano_heterodyne

def wavread(filename):
  """Read in audio data from a wav file.  Return d, sr."""
  # Read in wav file.
  file_handle = open(filename, 'rb')
  samplerate, wave_data = wav.read(file_handle)
  # Normalize short ints to floats in range [-1..1).
  data = np.asarray(wave_data, dtype=np.float32) / 32768.0
  return data, samplerate

def wavwrite(data, samplerate, filename):
  """Write a waveform to a WAV file."""
  wav.write(filename, samplerate, (32768.0 * data).astype(np.int16))

def read_and_trim(filename, duration=2.0, rel_threshold=0.015, abs_threshold=0.0006, sr=44100, channel=0, do_plot=False, pre_time=0.002):
    d, file_sr = wavread(filename)
    #print("d.shape=", d.shape)
    if len(d.shape) > 1:  # Stereo file
        if channel is None:
            d = np.mean(d, axis=-1)
        else:
            d = d[:, channel]
    assert file_sr == sr
    d = d - np.mean(d)
    threshold = np.maximum(np.max(np.abs(d)) * rel_threshold, abs_threshold)
    # Tight window at start?
    drop_initial_samps = np.maximum(0, -int(round(pre_time * sr)) + np.min(np.flatnonzero(np.abs(d) > threshold)))
    d_return = d[int(round(drop_initial_samps)) + np.arange(int(round(duration * sr)))]
    if do_plot:
        t = np.arange(len(d)) / sr
        plt.figure(figsize=(12, 4))
        plt.subplot(121)
        plt.plot(t[:50000], d[:50000])
        plt.ylim(threshold * np.array([-1, 1]))
        plt.subplot(122)
        plt.plot(t[:len(d_return)], d_return)
        plt.xlim([0, 0.1])
        print("drop initial", drop_initial_samps / sr)
    return d_return

def piano_filename(note='C', octave=4, strike='ff'):
    #filename = 'Piano.ff.D4.wav'
    directory = '/Users/dpwe/Downloads/uiowa-piano'
    #octave = 2
    #note = 'D'
    #octave = 4
    #note = 'Eb'
    #strike = 'ff'  # ['pp', 'mf', 'ff']:
    filename = os.path.join(directory, '.'.join(['Piano', strike, note + str(octave), 'wav']))
    return filename

filename = piano_filename('D', 4)
print(filename)
waveform = read_and_trim(filename)
_, sr = wavread(filename)
Audio(data=waveform.T, rate=sr)

In [ ]:
# Give it a 50ms fade-out
def fade_out(waveform, fade_end_sec=0.5, fade_duration_sec=0.05, sample_rate=44100):
    """Apply a fade-out to a waveform."""
    fade_end_samples = int(round(fade_end_sec * sample_rate))
    fade_duration_samples = int(round(fade_duration_sec * sample_rate))
    fade_start_samples = fade_end_samples - fade_duration_samples
    mask = np.ones_like(waveform)
    mask[fade_start_samples : fade_start_samples + fade_duration_samples] = np.linspace(1, 0, fade_duration_samples)
    mask[fade_start_samples + fade_duration_samples:] = 0
    return mask * waveform

fade_waveform = fade_out(waveform)
#plt.plot(fade_waveform)
Audio(data=fade_waveform.T, rate=sr)

In [ ]:
# Compose the 3s (time, note, vel) sequence from piano_examples.py

def emplace(main_waveform, waveform_to_add, start_time_sec, sample_rate=44100):
    """Add-in a waveform to a background at a given time, truncate as needed."""
    start_time_samples = int(round(start_time_sec * sample_rate))
    length_to_use = min(len(waveform_to_add), len(main_waveform) - start_time_samples)
    main_waveform[start_time_samples : start_time_samples + length_to_use] += waveform_to_add[:length_to_use]
    return main_waveform

sr = 44100
waveform = np.zeros(int(round(3.3 * sr)))

waveform = emplace(waveform, fade_out(read_and_trim(piano_filename('D', 4, 'pp')), 0.4), 0.05)
waveform = emplace(waveform, fade_out(read_and_trim(piano_filename('D', 4, 'mf')), 0.4), 0.45)
waveform = emplace(waveform, fade_out(read_and_trim(piano_filename('D', 4, 'ff')), 0.65), 0.85)

waveform = emplace(waveform, fade_out(read_and_trim(piano_filename('D', 2, 'mf')), 1.8), 1.5)
waveform = emplace(waveform, fade_out(read_and_trim(piano_filename('D', 6, 'ff')), 1.2), 2.1)

Audio(data=waveform.T, rate=sr)                    

In [ ]:
wavwrite(waveform, sr, '../sounds/piano_example_original.wav')

In [ ]:
filename = '../sounds/piano_example_original.wav'
waveform, sr = wavread(filename)
plt.figure(figsize=(16, 4))
spectrogram(waveform, sr=sr, f_max=8000, title='UIowa Samples')
Audio(data=waveform, rate=sr)

In [ ]:
filename = '../sounds//piano_example_juno_patch_7.wav'
waveform, sr = wavread(filename)
plt.figure(figsize=(16, 4))
spectrogram(waveform, sr=sr, f_max=8000, title='Juno-60 piano')
Audio(data=waveform, rate=sr)

In [ ]:
filename = '../sounds//piano_example_dx7_patch_137.wav'
waveform, sr = wavread(filename)
plt.figure(figsize=(16, 4))
spectrogram(waveform, sr=sr, f_max=8000, title='DX7 piano')
Audio(data=waveform, rate=sr)

In [ ]:
filename = '../sounds/piano_example_additive_fixed.wav'
waveform, sr = wavread(filename)
plt.figure(figsize=(16, 4))
spectrogram(waveform, sr=sr, f_max=8000, title='Fixed-envelope additive piano')
Audio(data=waveform, rate=sr)

In [ ]:
filename = '../sounds//piano_example_additive_interpolated.wav'
waveform, sr = wavread(filename)
plt.figure(figsize=(16, 4))
spectrogram(waveform, sr=sr, f_max=8000, title='Interpolated additive piano')
Audio(data=waveform, rate=sr)

In [ ]:
# Repeat original again
filename = '../sounds//piano_example_original.wav'
waveform, sr = wavread(filename)
plt.figure(figsize=(16, 4))
spectrogram(waveform, sr=sr, f_max=8000, title='UIowa Samples')
Audio(data=waveform, rate=sr)

In [ ]:
# Plot FFT to illustrate two-sided spectrum
from scipy import fft

def plotspec(d, n_fft=4096, sr=44100, f_max=6000, title=None, db_range=80):
    X = fft.fft(d[:n_fft] * np.hanning(n_fft))
    # FFT shift to put zero in the middle.
    fft_keep_points = int(round(f_max / sr * n_fft))
    X = np.hstack([X[-fft_keep_points:], X[:fft_keep_points]])
    freqs = np.arange(-fft_keep_points, fft_keep_points) * sr / n_fft / 1000
    db_X = dB(X)
    plt.plot(freqs, db_X)
    db_max = 10 * np.ceil(np.max(db_X) / 10)
    plt.ylim([db_max - db_range, db_max])
    plt.xlabel('freq / kHz')
    plt.ylabel('level / dB')
    plt.grid()
    if title:
        plt.title(title)

d = read_and_trim(piano_filename('D', 4, 'ff'))
# Skip first 0.2 s to exclude "thump"
plt.figure(figsize=(16, 4))
plotspec(d[int(round(0.2 * sr)):], title='D4.ff spectrum', f_max=3000)

In [ ]:
# Demodulate by multiplying by complex exponential
f0 = 293.665  # Nominal frequency of D4
d_demod = d * np.exp(-1j * 2 * np.pi * f0 * np.arange(len(d))/ sr)
plotspec(d_demod[int(round(0.2 * sr)):] , title='D4.ff shifted down by f0', f_max=3000)
f0_on_2 = f0 / 2000
plt.figure(figsize=(16, 4))
plt.plot([-f0_on_2, f0_on_2, f0_on_2, -f0_on_2, -f0_on_2], [38, 38, -38, -38, 38], '--r')

In [ ]:
# Low-pass demodulated waveform to recover harmonic envelope.

def demod_and_smooth(d, f0, harmonic=1, sr=44100):
    d_demod = d * np.exp(-1j * 2 * np.pi * f0 * harmonic * np.arange(len(d))/ sr)
    fundamental_period_samples = int(round(sr / f0))
    d_demod_smoo = np.convolve(d_demod, np.hanning(2 * fundamental_period_samples), 'same')
    return d_demod_smoo

t = np.arange(len(d)) / sr
plt.figure(figsize=(16, 6))

plt.subplot(211)
plt.plot(t, d)
plt.ylabel('amplitude')
plt.title('D4.ff waveform')

plt.subplot(212)
plt.plot(t, dB(demod_and_smooth(d, f0)))
plt.plot(t, dB(demod_and_smooth(d, f0, harmonic=2)))
plt.plot(t, dB(demod_and_smooth(d, f0, harmonic=3)))
plt.plot(t, dB(demod_and_smooth(d, f0, harmonic=4)))
plt.ylim([-15, 25])
plt.title('Extracted harmonic envelopes')
plt.xlabel('time / sec')
plt.ylabel('level / dB')
plt.legend(['%d - %.2f Hz' % (n, n * f0) for n in range(1, 5)])